<a href="https://colab.research.google.com/github/darwinyusef/20exHuggingFacePytorchTensorFlowSklearn/blob/master/ex2/TitanicMLv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy scikit-learn
!pip install imbalanced-learn


In [ ]:
import numpy as np
# Importamos las librerías necesarias
import pandas as pd  # Para manejar datos en forma de DataFrame
from sklearn.preprocessing import LabelEncoder, StandardScaler  # Para codificación y normalización
from sklearn.model_selection import train_test_split  # Para dividir los datos en entrenamiento y prueba
from sklearn.ensemble import RandomForestClassifier  # Modelo de clasificación basado en árboles
from sklearn.metrics import accuracy_score, confusion_matrix # Para evaluar el rendimiento del modelo
from imblearn.under_sampling import RandomUnderSampler  # Para balanceo de clases en el dataset
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Cargar datos
data = pd.read_csv('/content/titanic.csv')
data

In [ ]:
# Assuming the original column names are 'name' and 'fare' (case-sensitive)
titanic = data.drop(['name', 'Fare'], axis=1, errors='ignore')
# axis=1 specifies dropping columns, errors='ignore' prevents error if columns not found
titanic

In [ ]:
titanic.shape

In [ ]:
titanic.head()

In [ ]:
# @markdown **Limpieza de datos**
titanic.dropna(inplace=True)  # @markdown Eliminamos filas con valores nulos para evitar problemas en el entrenamiento

# @markdown **Codificación de variables categóricas**
# @markdown Convertimos la columna 'Sex' (que contiene valores 'male' y 'female') en valores numéricos (0 y 1)
titanic['Sex'] = LabelEncoder().fit_transform(titanic['Sex'])

# @markdown **Definimos las variables predictoras (X) y la variable objetivo (y)**
X = titanic.drop(columns=['Survived'])  # Eliminamos la columna 'Survived' para usar el resto como características
y = titanic['Survived']  # La columna 'Survived' será la variable objetivo (1 = sobrevivió, 0 = no sobrevivió)

# @markdown **Balanceo de clases con UnderSampling**
# @markdown Este paso es útil si la cantidad de sobrevivientes y no sobrevivientes está desbalanceada
undersampler = RandomUnderSampler(random_state=42)  # @markdown Creamos un objeto para realizar el submuestreo
X, y = undersampler.fit_resample(X, y)  # @markdown Aplicamos el submuestreo para equilibrar las clases

# @markdown **División del dataset en entrenamiento y prueba**
# @markdown Separamos el conjunto de datos en un 80% para entrenamiento y 20% para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# @markdown **Normalización de los datos**
scaler = StandardScaler()  # @markdown Creamos un objeto StandardScaler para normalizar los datos
X_train = scaler.fit_transform(X_train)  # @markdown Ajustamos el escalador en los datos de entrenamiento y los transformamos
X_test = scaler.transform(X_test)  # @markdown Usamos el mismo escalador para transformar los datos de prueba

# @markdown **Entrenamiento del modelo**
model = RandomForestClassifier(n_estimators=100, random_state=42)  # @markdown Creamos el modelo de bosque aleatorio con 100 árboles
model.fit(X_train, y_train)  # @markdown Entrenamos el modelo con los datos de entrenamiento

# @markdown**Predicción en los datos de prueba**
y_pred = model.predict(X_test)  # @markdown Generamos predicciones con el modelo entrenado

# @markdown **Evaluación del modelo**
accuracy = accuracy_score(y_test, y_pred)  # @markdown Calculamos la precisión comparando predicciones con valores reales
print(f'Precisión del modelo: {accuracy:.4f}') # @markdown Mostramos la precisión del modelo en la consola


In [ ]:
# Importancia de características
feature_importances = model.feature_importances_
feature_names = X.columns
plt.figure(figsize=(8,5))
sns.barplot(x=feature_importances, y=feature_names)
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.title('Importancia de las Características')
plt.show()

# Matriz de Confusión
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['No Sobrevivió', 'Sobrevivió'], yticklabels=['No Sobrevivió', 'Sobrevivió'])
plt.xlabel('Predicción')
plt.ylabel('Real')
plt.title('Matriz de Confusión')
plt.show()


In [ ]:
# Obtener la importancia de características del modelo
importances = model.feature_importances_
feature_names = X.columns

# Crear la figura
fig, axes = plt.subplots(2, 1, figsize=(8, 12))

# Gráfica de Importancia de Características
sns.barplot(x=importances, y=feature_names, ax=axes[0], palette="Blues_r", edgecolor="black")
axes[0].set_xlabel("Importancia", fontsize=12)
axes[0].set_ylabel("Características", fontsize=12)
axes[0].set_title("Importancia de las Características", fontsize=14, fontweight="bold")

# Agregar etiquetas de valores en cada barra
for i, v in enumerate(importances):
    axes[0].text(v + 0.01, i, f"{v:.2f}", color="black", va="center", fontsize=11)

# Matriz de Confusión
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_percentage = conf_matrix.astype("float") / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

# Crear mapa de calor de la matriz de confusión
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", linewidths=1, linecolor="black",
            xticklabels=["No Sobrevivió", "Sobrevivió"], yticklabels=["No Sobrevivió", "Sobrevivió"], ax=axes[1])
axes[1].set_xlabel("Predicción", fontsize=12)
axes[1].set_ylabel("Real", fontsize=12)
axes[1].set_title("Matriz de Confusión", fontsize=14, fontweight="bold")

# Ajustar el diseño
plt.tight_layout()
plt.show()